In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 定义 AlexNet
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 使用 CIFAR-10 数据集作为例子
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=10).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练
for epoch in range(5):  # 可调整 epochs
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(train_loader):.4f}")

# 测试
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")


100%|██████████| 170498071/170498071 [00:03<00:00, 48322309.12it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/5], Loss: 1.7171
Epoch [2/5], Loss: 1.3820
Epoch [3/5], Loss: 1.2384
Epoch [4/5], Loss: 1.1311
Epoch [5/5], Loss: 1.0511
Accuracy: 65.49%


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
from tqdm import tqdm

# 配置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据预处理和增强
transform_train = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # CIFAR-10标准化参数
])

transform_test = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# 加载数据集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

class ModernAlexNet(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.5):
        super(ModernAlexNet, self).__init__()
        self.features = nn.Sequential(
            # 第一个卷积块
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout2d(0.25),
            
            # 第二个卷积块
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout2d(0.25),
            
            # 第三个卷积块
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            
            # 第四个卷积块
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            # 第五个卷积块
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout2d(0.25)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train_one_epoch(model, train_loader, criterion, optimizer, device, scheduler=None):
    model.train()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    pbar = tqdm(train_loader, desc='Training')
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)
        
        # 混合精度训练
        with torch.cuda.amp.autocast(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # 计算准确率
        _, predicted = outputs.max(1)
        correct = predicted.eq(labels).sum().item()
        acc = 100. * correct / labels.size(0)
        
        losses.update(loss.item(), images.size(0))
        top1.update(acc, images.size(0))
        
        # 优化器步骤
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        if scheduler is not None:
            scheduler.step()
        
        # 更新进度条
        pbar.set_postfix({
            'loss': f'{losses.avg:.4f}',
            'acc': f'{top1.avg:.2f}%'
        })
    
    return losses.avg, top1.avg

def evaluate(model, test_loader, criterion, device):
    model.eval()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            _, predicted = outputs.max(1)
            correct = predicted.eq(labels).sum().item()
            acc = 100. * correct / labels.size(0)
            
            losses.update(loss.item(), images.size(0))
            top1.update(acc, images.size(0))
    
    return losses.avg, top1.avg

def main():
    # 初始化模型
    model = ModernAlexNet().to(device)
    criterion = nn.CrossEntropyLoss()
    
    # 使用AdamW优化器
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    
    # 使用 OneCycleLR 调度器
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=0.001,
        epochs=50, steps_per_epoch=len(train_loader)
    )
    
    # 训练配置
    n_epochs = 50
    best_acc = 0
    
    # 训练循环
    for epoch in range(n_epochs):
        start_time = time.time()
        
        # 训练阶段
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, scheduler
        )
        
        # 评估阶段
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        # 保存最佳模型
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_acc': best_acc,
            }, 'best_alexnet_cifar10.pth')
        
        # 打印训练信息
        epoch_time = time.time() - start_time
        print(f"\nEpoch [{epoch+1}/{n_epochs}]")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print(f"Time: {epoch_time:.2f}s")
        print("-" * 60)
    
    print(f"Best Test Accuracy: {best_acc:.2f}%")

if __name__ == "__main__":
    main()

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


Training:   0%|          | 0/391 [00:00<?, ?it/s]/tmp/ipykernel_23/2627079015.py:131: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Training: 100%|██████████| 391/391 [01:37<00:00,  4.00it/s, loss=1.9172, acc=28.67%]



Epoch [1/50]
Train Loss: 1.9172, Train Acc: 28.67%
Test Loss: 1.5585, Test Acc: 42.27%
Time: 110.40s
------------------------------------------------------------


Training: 100%|██████████| 391/391 [01:43<00:00,  3.79it/s, loss=1.6336, acc=39.73%]



Epoch [2/50]
Train Loss: 1.6336, Train Acc: 39.73%
Test Loss: 1.4163, Test Acc: 47.76%
Time: 116.41s
------------------------------------------------------------


...


Epoch [48/50]
Train Loss: 0.3844, Train Acc: 86.53%
Test Loss: 0.3346, Test Acc: 88.51%
Time: 124.88s
------------------------------------------------------------


Training: 100%|██████████| 391/391 [01:53<00:00,  3.44it/s, loss=0.3904, acc=86.59%]



Epoch [49/50]
Train Loss: 0.3904, Train Acc: 86.59%
Test Loss: 0.3329, Test Acc: 88.56%
Time: 125.52s
------------------------------------------------------------


Training: 100%|██████████| 391/391 [01:53<00:00,  3.44it/s, loss=0.3873, acc=86.36%]



Epoch [50/50]
Train Loss: 0.3873, Train Acc: 86.36%
Test Loss: 0.3343, Test Acc: 88.57%
Time: 125.66s
------------------------------------------------------------
Best Test Accuracy: 88.63%


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from tqdm import tqdm
import numpy as np

# 设置随机种子
def set_seed(seed=42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

# 配置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据预处理和数据增强
transform_train = transforms.Compose([
    transforms.Resize(256),  # 先调整为较大尺寸
    transforms.RandomCrop(227),  # 随机裁剪到目标尺寸
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

class ModernAlexNet(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.5):
        super(ModernAlexNet, self).__init__()
        
        # 特征提取器
        self.features = nn.Sequential(
            # Conv1
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout2d(0.2),
            
            # Conv2
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout2d(0.2),
            
            # Conv3
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
            
            # Conv4
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
            
            # Conv5
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout2d(0.2)
        )
        
        # 计算卷积层输出大小
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        # 分类器
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            nn.Dropout(dropout_rate),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, num_classes)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

def train_epoch(model, train_loader, criterion, optimizer, device, scheduler=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        if scheduler is not None:
            scheduler.step()
        
        # 统计
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # 更新进度条
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss / len(train_loader), 100. * correct / total

def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(test_loader), 100. * correct / total

def main():
    # 设置随机种子
    set_seed(42)
    
    # 加载数据集
    train_dataset = datasets.CIFAR10(
        root='./data', train=True, transform=transform_train, download=True)
    test_dataset = datasets.CIFAR10(
        root='./data', train=False, transform=transform_test, download=True)
    
    train_loader = DataLoader(
        train_dataset, batch_size=64, shuffle=True,
        num_workers=2, pin_memory=True)
    test_loader = DataLoader(
        test_dataset, batch_size=128, shuffle=False,
        num_workers=2, pin_memory=True)
    
    # 初始化模型和训练组件
    model = ModernAlexNet(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    
    # 学习率调度
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.001,
        epochs=30,
        steps_per_epoch=len(train_loader),
        pct_start=0.1
    )
    
    # 训练循环
    best_acc = 0
    epochs = 30
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        start_time = time.time()
        
        # 训练
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device, scheduler)
        
        # 评估
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        # 保存最佳模型
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_acc': best_acc,
            }, 'best_alexnet_cifar10.pth')
        
        # 打印训练信息
        epoch_time = time.time() - start_time
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print(f"Time: {epoch_time:.2f}s")
        print("-" * 60)
    
    print(f"Best Test Accuracy: {best_acc:.2f}%")

if __name__ == "__main__":
    main()

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified

Epoch 1/30


Training: 100%|██████████| 782/782 [02:13<00:00,  5.84it/s, loss=2.1432, acc=32.58%]


Train Loss: 1.8393, Train Acc: 32.58%
Test Loss: 1.4404, Test Acc: 46.78%
Time: 147.43s
------------------------------------------------------------

Epoch 2/30


Training: 100%|██████████| 782/782 [02:06<00:00,  6.17it/s, loss=1.4615, acc=44.94%]


Train Loss: 1.5272, Train Acc: 44.94%
Test Loss: 1.2453, Test Acc: 54.33%
Time: 140.31s
------------------------------------------------------------

Epoch 3/30


...

Train Loss: 0.4571, Train Acc: 84.16%
Test Loss: 0.3650, Test Acc: 87.66%
Time: 146.86s
------------------------------------------------------------

Epoch 29/30


Training: 100%|██████████| 782/782 [02:12<00:00,  5.89it/s, loss=0.7366, acc=84.33%]


Train Loss: 0.4515, Train Acc: 84.33%
Test Loss: 0.3650, Test Acc: 87.70%
Time: 146.32s
------------------------------------------------------------

Epoch 30/30


Training: 100%|██████████| 782/782 [02:12<00:00,  5.91it/s, loss=0.3171, acc=84.22%]


Train Loss: 0.4545, Train Acc: 84.22%
Test Loss: 0.3672, Test Acc: 87.61%
Time: 144.93s
------------------------------------------------------------
Best Test Accuracy: 87.70%


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import time
from tqdm import tqdm

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据预处理和增强
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# 加载数据集
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 加载预训练的 ResNet50
model = models.resnet50(pretrained=True)

# 修改最后一层以适应 CIFAR-10（10 类）
model.fc = nn.Linear(model.fc.in_features, 10)

# 冻结所有层，除了最后一层
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# 设置优化器和损失函数
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 训练过程
epochs = 30
best_acc = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc="Training")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 统计
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100.*correct/total:.2f}%'})

    print(f"Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {100.*correct/total:.2f}%")

    # 评估
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Acc: {100.*correct/total:.2f}%")

    # 保存最佳模型
    if correct / total > best_acc:
        best_acc = correct / total
        torch.save(model.state_dict(), 'best_model.pth')

print(f"Best Test Accuracy: {best_acc:.2f}%")


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 179MB/s]


Epoch 1/30


Training: 100%|██████████| 782/782 [04:37<00:00,  2.82it/s, loss=0.8799, acc=70.94%]


Train Loss: 0.8913, Train Acc: 70.94%
Test Loss: 0.7026, Test Acc: 75.88%
Epoch 2/30


...

Train Loss: 0.5401, Train Acc: 81.37%
Test Loss: 0.5425, Test Acc: 81.40%
Epoch 29/30


Training: 100%|██████████| 782/782 [04:35<00:00,  2.84it/s, loss=0.7423, acc=81.63%]


Train Loss: 0.5336, Train Acc: 81.63%
Test Loss: 0.5687, Test Acc: 80.94%
Epoch 30/30


Training: 100%|██████████| 782/782 [04:35<00:00,  2.84it/s, loss=0.8613, acc=81.50%]


Train Loss: 0.5336, Train Acc: 81.50%
Test Loss: 0.5209, Test Acc: 81.65%
Best Test Accuracy: 0.82%


# vit_base_patch16_224',

In [5]:
pip install timm torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from tqdm import tqdm
import timm
import numpy as np

# 设置随机种子
def set_seed(seed=42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

# 配置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据预处理和数据增强
transform_train = transforms.Compose([
    transforms.Resize(256),  # 先调整为较大尺寸
    transforms.RandomCrop(224),  # 随机裁剪到目标尺寸
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# 加载 CIFAR-10 数据集
train_dataset = datasets.CIFAR10(
    root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(
    root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True,
    num_workers=2, pin_memory=True)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False,
    num_workers=2, pin_memory=True)

# 使用 ViT 模型（如果 CaiT 模型不可用）
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 10)  # 修改输出层，适应 CIFAR-10 数据集（10 类）

model.to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

# 学习率调度器
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.001,
    epochs=30,
    steps_per_epoch=len(train_loader),
    pct_start=0.1
)

# 设置随机种子
set_seed(42)

# 训练和评估函数
def train_epoch(model, train_loader, criterion, optimizer, device, scheduler=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        if scheduler is not None:
            scheduler.step()
        
        # 统计
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # 更新进度条
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss / len(train_loader), 100. * correct / total

def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(test_loader), 100. * correct / total

# 训练过程
def main():
    best_acc = 0
    epochs = 3
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        start_time = time.time()
        
        # 训练
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device, scheduler)
        
        # 评估
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        # 保存最佳模型
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_acc': best_acc,
            }, 'best_cait_model.pth')
        
        # 打印训练信息
        epoch_time = time.time() - start_time
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print(f"Time: {epoch_time:.2f}s")
        print("-" * 60)
    
    print(f"Best Test Accuracy: {best_acc:.2f}%")

if __name__ == "__main__":
    main()


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]


Epoch 1/3


Training: 100%|██████████| 782/782 [28:10<00:00,  2.16s/it, loss=0.2711, acc=89.91%]


Train Loss: 0.3131, Train Acc: 89.91%
Test Loss: 0.4209, Test Acc: 85.81%
Time: 1800.78s
------------------------------------------------------------

Epoch 2/3


Training: 100%|██████████| 782/782 [28:10<00:00,  2.16s/it, loss=1.9200, acc=70.94%]


Train Loss: 0.8530, Train Acc: 70.94%
Test Loss: 1.5533, Test Acc: 45.63%
Time: 1798.61s
------------------------------------------------------------

Epoch 3/3


Training: 100%|██████████| 782/782 [28:06<00:00,  2.16s/it, loss=1.6653, acc=43.43%]


Train Loss: 1.5640, Train Acc: 43.43%
Test Loss: 1.6118, Test Acc: 39.96%
Time: 1794.38s
------------------------------------------------------------
Best Test Accuracy: 85.81%


# The end